# Degradation and Acceleration Factors
This tool will provide a simple method for estimating degradation and for calculating acceleration factors. It interfaces with the degradation database to simplify acquisition of the degradation parameters.

**Requirements**:
- compatible weather file (e.g., PSM3, TMY3, EPW...)
- Accelerated testing chamber parameters
    - chamber irradiance [W/m^2]
    - chamber temperature [C]
    - chamber humidity [%]
    - & etc.
- Activation energies for test material [kJ/mol]
- Other degradation parameters

**Objectives**:
1. Read in the weather data
2. Gather basic degradation modeling data for a material of interest
3. Calculate absolute degradation rate
4. Run Monte Carlo simulation at a single site
5. Generate chamber or field data for environmental comparison
6. Calculate degradation acceleration factor of field location to chamber (or another location)
7. Run Monte Carlo simulation of the acceleration factor for a single site
8. Select a region of interest and specific data points
9. Produce a map of acceleration factors for a region

In [1]:
# if running on google colab, uncomment the next line and execute this cell to install the dependencies and prevent "ModuleNotFoundError" in later cells:
# !pip install pvdeg==0.4.2

In [1]:
import os
import pvdeg
import pandas as pd
from pvdeg import DATA_DIR
import json
import numpy as np
from IPython.display import display, Math

import pvlib
print(pvlib.__version__)
from pvlib import iotools

0.13.1


In [2]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("pvdeg version ", pvdeg.__version__)
print(DATA_DIR)

Working on a  Windows 10
Python version  3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.4
pvdeg version  0.1.dev1152+gfbac6aa1f.d20250917
C:\Users\mprillim\sam_dev\PVDegradationTools\pvdeg\data


## 1. Read In the Weather Data

The function has these minimum requirements when using a weather data file:
- Weather data containing (at least) DNI, DHI, GHI, Temperature, RH, and Wind-Speed data at module level.
- Site meta-data containing (at least) latitude, longitude, and time zone

Alternatively one may can get meterological data from the NSRDB or PVGIS with just the longitude and latitude. This function for the NSRDB (via NSRDB 'PSM3') works primarily for most of North America and South America. PVGIS works for most of the rest of the world (via SARAH 'PVGIS'). See the tutorial "Weather Database Access.ipynb" tutorial on PVdeg or Jensen et al. https://doi.org/10.1016/j.solener.2023.112092 for satellite coverage information.

In [3]:
# Get data from a supplied data file (Do not use the next box of code if using your own file)
weather_file = os.path.join(DATA_DIR, 'psm3_demo.csv')
weather_df, meta = pvdeg.weather.read(weather_file,'csv',find_meta=False)
print(weather_df)
print(meta)

meta = pvdeg.weather.map_meta(meta)
meta = pvdeg.weather.find_metadata(meta)
print(meta)

                           Year  Month  Day  Hour  Minute  dni  dhi  ghi  \
1999-01-01 00:30:00-07:00  1999      1    1     0      30  0.0  0.0  0.0   
1999-01-01 01:30:00-07:00  1999      1    1     1      30  0.0  0.0  0.0   
1999-01-01 02:30:00-07:00  1999      1    1     2      30  0.0  0.0  0.0   
1999-01-01 03:30:00-07:00  1999      1    1     3      30  0.0  0.0  0.0   
1999-01-01 04:30:00-07:00  1999      1    1     4      30  0.0  0.0  0.0   
...                         ...    ...  ...   ...     ...  ...  ...  ...   
1999-12-31 19:30:00-07:00  1999     12   31    19      30  0.0  0.0  0.0   
1999-12-31 20:30:00-07:00  1999     12   31    20      30  0.0  0.0  0.0   
1999-12-31 21:30:00-07:00  1999     12   31    21      30  0.0  0.0  0.0   
1999-12-31 22:30:00-07:00  1999     12   31    22      30  0.0  0.0  0.0   
1999-12-31 23:30:00-07:00  1999     12   31    23      30  0.0  0.0  0.0   

                           temp_air  dew_point  wind_speed  relative_humidity  
1999-01

In [4]:
# This routine will get a meteorological dataset from anywhere in the world where it is available 
#weather_id = (24.7136, 46.6753) #Riyadh, Saudi Arabia
#weather_id = (35.6754, 139.65) #Tokyo, Japan
#weather_id = (-43.52646, 172.62165) #Christchurch, New Zealand
#weather_id = (64.84031, -147.73836) #Fairbanks, Alaska
#weather_id = (65.14037, -21.91633) #Reykjavik, Iceland
weather_id = (33.4152, -111.8315) #Mesa, Arizona
#weather_id = (0,0) # Somewhere else you are interested in.
weather_arg = {
    "api_key": "x49AVPQbyYBOFkjtj91XZf8CSWq3NWza2sajAGT8", 
    "email": "mprillim@nrel.gov",
    "names": "tmy",
    "attributes": [],
    "map_variables": True,
    "geospatial": False,
    "find_meta": True
}
weather_df, meta = pvdeg.weather.get_anywhere(id=weather_id, **weather_arg)
print(weather_df)
print(meta)
display(weather_df)

                           temp_air  relative_humidity    ghi     dni   dhi  \
time(UTC)                                                                     
1990-01-01 00:00:00+00:00      7.48              66.54   10.0    0.00  10.0   
1990-01-01 01:00:00+00:00      7.14              67.89    0.0    0.00   0.0   
1990-01-01 02:00:00+00:00      6.79              69.23    0.0    0.00   0.0   
1990-01-01 03:00:00+00:00      6.45              70.58    0.0    0.00   0.0   
1990-01-01 04:00:00+00:00      6.11              71.92    0.0    0.00   0.0   
...                             ...                ...    ...     ...   ...   
1990-12-31 19:00:00+00:00      9.20              59.82  592.0  945.23  76.0   
1990-12-31 20:00:00+00:00      8.85              61.16  593.0  946.11  76.0   
1990-12-31 21:00:00+00:00      8.51              62.51  530.0  923.10  73.0   
1990-12-31 22:00:00+00:00      8.17              63.85  407.0  865.21  65.0   
1990-12-31 23:00:00+00:00      7.82              65.

,temp_air,relative_humidity,ghi,dni,dhi,IR(h),wind_speed,wind_direction,pressure
time(UTC),,,,,,,,,
1990-01-01 00:00:00+00:00,7.48,66.54,10.0,0.00,10.0,269.07,1.59,156.0,96989.0
1990-01-01 01:00:00+00:00,7.14,67.89,0.0,0.00,0.0,269.14,1.54,110.0,97008.0
1990-01-01 02:00:00+00:00,6.79,69.23,0.0,0.00,0.0,269.22,1.49,106.0,97036.0
1990-01-01 03:00:00+00:00,6.45,70.58,0.0,0.00,0.0,269.29,1.44,140.0,97103.0
1990-01-01 04:00:00+00:00,6.11,71.92,0.0,0.00,0.0,269.36,1.39,189.0,97103.0
...,...,...,...,...,...,...,...,...,...
1990-12-31 19:00:00+00:00,9.20,59.82,592.0,945.23,76.0,268.72,1.84,255.0,97552.0
1990-12-31 20:00:00+00:00,8.85,61.16,593.0,946.11,76.0,268.79,1.79,259.0,97466.0
1990-12-31 21:00:00+00:00,8.51,62.51,530.0,923.10,73.0,268.86,1.74,271.0,97447.0


#### POA Irradiance
Next we need to calculate the stress parameters including temperature and humidity. We start with POA irradiance.
Irradiance_kwarg governs the array orientation for doing the POA calculations. 
It is defaulted to a north-south single axis tracking. A fixed tilt set of parameters is included but is blocked out. 
Look in spectral.py and/or PVLib here for 1-axis kwargs, 
https://pvlib-python.readthedocs.io/en/v0.7.2/generated/pvlib.tracking.singleaxis.html#pvlib.tracking.singleaxis  
and for fixed tilt,  
https://pvlib-python.readthedocs.io/en/v0.7.2/generated/pvlib.irradiance.gti_dirint.html?highlight=poa .  

In [ ]:
#irradiance_kwarg ={
    #"tilt": None,
    #"azimuth": None,
    #"module_mount": 'fixed'}
irradiance_kwarg ={
    "axis_tilt": None,
    "axis_azimuth": None,
    # "albedo": 0.2, #If there is no albedo in weather file, set a default value here. 
    "module_mount": 'single_axis'}
poa_df = pvdeg.spectral.poa_irradiance(weather_df=weather_df, meta=meta, **irradiance_kwarg)
#display(poa_df)

The array axis_azimuth was not provided, therefore an azimuth of  180.0 was used.
The array axis_tilt was not provided, therefore an axis tilt of 0° was used.


In [5]:
print(weather_df.index.duplicated().any())
print(weather_df.loc[:,weather_df.columns.duplicated()])

False
Empty DataFrame
Columns: []
Index: [1999-01-01 00:30:00-07:00, 1999-01-01 01:30:00-07:00, 1999-01-01 02:30:00-07:00, 1999-01-01 03:30:00-07:00, 1999-01-01 04:30:00-07:00, 1999-01-01 05:30:00-07:00, 1999-01-01 06:30:00-07:00, 1999-01-01 07:30:00-07:00, 1999-01-01 08:30:00-07:00, 1999-01-01 09:30:00-07:00, 1999-01-01 10:30:00-07:00, 1999-01-01 11:30:00-07:00, 1999-01-01 12:30:00-07:00, 1999-01-01 13:30:00-07:00, 1999-01-01 14:30:00-07:00, 1999-01-01 15:30:00-07:00, 1999-01-01 16:30:00-07:00, 1999-01-01 17:30:00-07:00, 1999-01-01 18:30:00-07:00, 1999-01-01 19:30:00-07:00, 1999-01-01 20:30:00-07:00, 1999-01-01 21:30:00-07:00, 1999-01-01 22:30:00-07:00, 1999-01-01 23:30:00-07:00, 1999-01-02 00:30:00-07:00, 1999-01-02 01:30:00-07:00, 1999-01-02 02:30:00-07:00, 1999-01-02 03:30:00-07:00, 1999-01-02 04:30:00-07:00, 1999-01-02 05:30:00-07:00, 1999-01-02 06:30:00-07:00, 1999-01-02 07:30:00-07:00, 1999-01-02 08:30:00-07:00, 1999-01-02 09:30:00-07:00, 1999-01-02 10:30:00-07:00, 1999-01-02 11


#### Get Spectrally Resolved Irradiance Data
This first set of commands will calculate spectrally resolved irradiance data. This may or may not be needed for a given degradation model and can be skipped here. 

In [ ]:
# this whole block needs to be replaced with call to calculate spectrally resolved irradiance.

from pvdeg import TEST_DATA_DIR
INPUT_SPECTRA = os.path.join(TEST_DATA_DIR, r"spectra_pytest.csv")
data = pd.read_csv(INPUT_SPECTRA)
#display(data)
print(INPUT_SPECTRA)

#Test function 
#cusotm_albedo['Summer']
#custom_albedo['Winter']
# custom_albedo['Snow']
#defaults - Grass, Dry Grass, Snow
#Flexible to add complexity later
#merge in development branch changes
#KGPCY Python package
custom_albedo_summer = 'A006'
custom_albedo_winter = 'A008'
custom_albedo_snow = { #required: startDate, wavelength (if len(albedo) > 1), albedo, isSnow defaults to False
    "data_entry_person": "Michael Kempe",
    "date_entered": "7/28/2025",
    "DOI": "10.3390/ijerph15071507",
    "source_title": "Ultraviolet Radiation Albedo and Reflectance in Review: The Influence to Ultraviolet Exposure in Occupational Settings",
    "authors": "Joanna Turner, Alfio V. Parisi",
    "reference": "Turner J, Parisi AV. Ultraviolet Radiation Albedo and Reflectance in Review: The Influence to Ultraviolet Exposure in Occupational Settings. Int J Environ Res Public Health. 2018 Jul 17;15(7):1507.",
    "keywords": "snow, ground",
    "months": "1,2,3,10,11,12",
    "startDate": "January 1", #Day of Year? 0-365 
    "DayOfYear": 1, #Hour or Day of Year? 1-8760
    "isSnow": True,
    "comments": "Data is emperically extrapolated from 280 nm to 297 nm. Data extracted from Turner et al. Figure 1 as a reference to Doda & Green Snow-Ground. Doda D., Green A. Surface Reflectance Measurements in the UV from an Airborne Platform. Part 1. Appl. Opt. 1980;19:2140-2145. doi: 10.1364/AO.19.002140. Doda D., Green A. Surface Reflectance Measurements in the Ultraviolet from an Airborne Platform. Part 2. Appl. Opt. 1981;20:636-642. doi: 10.1364/AO.20.000636.",
    "wavelength": "280, 297.32034, 300.02435, 301.8514, 305.79782, 310.10962, 313.21558, 317.4543, 322.86237, 329.9513, 331.19366, 339.89038, 343.06943, 350.34103, 360.02435, 369.96347, 380.3776, 386.77222, 390.0609, 400.14615",
    "albedo": "20, 29.515152, 28.30303, 29.454546, 28.90909, 34.696968, 36.757576, 39.363636, 39.21212, 38.60606, 41.272728, 40.909092, 42.242424, 42.21212, 40.575756, 43.21212, 43.090908, 43.454544, 43.60606, 39.757576"
}
#Startdate, albedo, wavelength -> then next one + boolean logic for snow ()
custom_albedo_snow = {}
custom_albedo_dict = {}
custom_albedo_dict['Summer'] = custom_albedo_summer
custom_albedo_dict['Winter'] = custom_albedo_winter
custom_albedo_dict['Snow'] = custom_albedo_snow
#Can have as many as you want, but need to define the start date and if snow or not.
# custom_albedo_snow 
spectra_folder = 'spectra' #If you have already pulled the spectra from SMARTS, pass the folder path to avoid going through the donwload process again. 
# spectra_folder = None
wavelengths = np.arange(280, 400, 25)  # Example wavelengths from 280 nm to 400 nm in steps of 25 nm
data = pvdeg.spectral.spectrally_resolved_irradiance(weather_df=weather_df, meta=meta, wavelengths=wavelengths, frontResultsOnly=None,
                                                     spectra_folder=spectra_folder, **irradiance_kwarg)

#return front, back, or both (True, False, None)
#bool frontResultsonly = True for front only
#separate columns for front and back irradiance: spectra_front: etc. , spectra_back: etc. (see spectra_pytest.csv)
#Check albedo boolean snow, winter non-snow, summer non-snow


C:\Users\mprillim\sam_dev\PVDegradationTools\tests\data\spectra_pytest.csv
path = c:\Users\mprillim\sam_dev\PVDegradationTools\tutorials_and_tools\tutorials_and_tools
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for center labeled data, at exact timestamp in input Weather File
<class 'bifacial_radiance.main.MetObj'>.metadata:
{'latitude': 39.73, 'longitude': -105.18, 'elevation': 1820.0, 'timezone': -7.0, 'city': '-', 'label': 'center'}
<class 'bifacial_radiance.main.MetObj'>.tmydata:
 <class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4301 entries, 1999-01-01 08:30:00-07:00 to 1999-12-31 16:30:00-07:00
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ghi         4301 non-null   float64
 1   dhi         4301 non-null   floa

Generating Spectral TMYs: 100%|███████████████████████████████████████| 5/5 [00:30<00:00,  6.19s/it]


                             GHI    DHI    DNI  DryBulb  Wspd
1999-01-01 08:30:00-07:00   76.0   65.0   65.0      1.0   4.7
1999-01-01 09:30:00-07:00  246.0   93.0  503.0      2.0   6.3
1999-01-01 10:30:00-07:00  355.0  109.0  617.0      3.0   7.0
1999-01-01 11:30:00-07:00  385.0  161.0  497.0      4.0   6.8
1999-01-01 12:30:00-07:00  128.0  128.0    0.0      4.0   6.5
...                          ...    ...    ...      ...   ...
1999-12-31 12:30:00-07:00   20.0   20.0    0.0      9.0   1.6
1999-12-31 13:30:00-07:00  354.0  125.0  562.0      8.0   0.5
1999-12-31 14:30:00-07:00  263.0   94.0  530.0      7.0   0.4
1999-12-31 15:30:00-07:00  159.0   38.0  636.0      5.0   1.0
1999-12-31 16:30:00-07:00   21.0   21.0    0.0      4.0   1.3

[4301 rows x 5 columns]
single_axis
The array axis_azimuth was not provided, therefore an azimuth of 180.0 was used.
The array axis_tilt was not provided, therefore an axis tilt of 0° was used.
                           Year  Month  Day  Hour  Minute  DN

100%|██████████| 4301/4301 [00:19<00:00, 222.07it/s]


Finished
Composite Data 1:                             date    DNI    DHI  albedo  decHRs         ghi  \
0     1999-01-01 08:30:00-07:00   65.0   65.0    0.62     8.0   80.701880   
1     1999-01-01 09:30:00-07:00  503.0   93.0    0.62     9.0  272.448214   
2     1999-01-01 10:30:00-07:00  617.0  109.0    0.62    10.0  374.707417   
3     1999-01-01 11:30:00-07:00  497.0  161.0    0.62    11.0  388.725056   
4     1999-01-01 12:30:00-07:00    0.0  128.0    0.62    12.0  128.000000   
...                         ...    ...    ...     ...     ...         ...   
4168  1999-12-31 11:30:00-07:00  985.0   54.0    0.62    11.0  503.862717   
4169  1999-12-31 12:30:00-07:00    0.0   20.0    0.62    12.0   20.000000   
4170  1999-12-31 13:30:00-07:00  562.0  125.0    0.62    13.0  331.676261   
4171  1999-12-31 14:30:00-07:00  530.0   94.0    0.62    14.0  230.625571   
4172  1999-12-31 15:30:00-07:00  636.0   38.0    0.62    15.0  109.989149   

            inc        zen         azm  pvFront

...

100%|██████████| 4301/4301 [00:17<00:00, 251.48it/s]


Finished
Composite Data 4:                             date    DNI    DHI  albedo  decHRs         ghi  \
0     1999-01-01 08:30:00-07:00   65.0   65.0    0.62     8.0   80.701880   
1     1999-01-01 09:30:00-07:00  503.0   93.0    0.62     9.0  272.448214   
2     1999-01-01 10:30:00-07:00  617.0  109.0    0.62    10.0  374.707417   
3     1999-01-01 11:30:00-07:00  497.0  161.0    0.62    11.0  388.725056   
4     1999-01-01 12:30:00-07:00    0.0  128.0    0.62    12.0  128.000000   
...                         ...    ...    ...     ...     ...         ...   
4168  1999-12-31 11:30:00-07:00  985.0   54.0    0.62    11.0  503.862717   
4169  1999-12-31 12:30:00-07:00    0.0   20.0    0.62    12.0   20.000000   
4170  1999-12-31 13:30:00-07:00  562.0  125.0    0.62    13.0  331.676261   
4171  1999-12-31 14:30:00-07:00  530.0   94.0    0.62    14.0  230.625571   
4172  1999-12-31 15:30:00-07:00  636.0   38.0    0.62    15.0  109.989149   

            inc        zen         azm  pvFront

100%|██████████| 4301/4301 [00:19<00:00, 220.01it/s]


Finished
Composite Data 5:                             date    DNI    DHI  albedo  decHRs         ghi  \
0     1999-01-01 08:30:00-07:00   65.0   65.0    0.62     8.0   80.701880   
1     1999-01-01 09:30:00-07:00  503.0   93.0    0.62     9.0  272.448214   
2     1999-01-01 10:30:00-07:00  617.0  109.0    0.62    10.0  374.707417   
3     1999-01-01 11:30:00-07:00  497.0  161.0    0.62    11.0  388.725056   
4     1999-01-01 12:30:00-07:00    0.0  128.0    0.62    12.0  128.000000   
...                         ...    ...    ...     ...     ...         ...   
4168  1999-12-31 11:30:00-07:00  985.0   54.0    0.62    11.0  503.862717   
4169  1999-12-31 12:30:00-07:00    0.0   20.0    0.62    12.0   20.000000   
4170  1999-12-31 13:30:00-07:00  562.0  125.0    0.62    13.0  331.676261   
4171  1999-12-31 14:30:00-07:00  530.0   94.0    0.62    14.0  230.625571   
4172  1999-12-31 15:30:00-07:00  636.0   38.0    0.62    15.0  109.989149   

            inc        zen         azm  pvFront

c:\users\mprillim\sam_dev\bifacialvf\bifacialvf\bifacialvf.py:958: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  composite_data['G_' + str(header) + '_DHI_' + str(x)] = composite_data['Avg_Row' + str(fb) + 'GTI_DHIDirect'] / Sum_DHI.mask(Sum_DHI == 0) * weather_df['DHI_0' + str(x)]
c:\users\mprillim\sam_dev\bifacialvf\bifacialvf\bifacialvf.py:960: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  composite_data['G_' + str(header) + '_DNI_reflected_' + str(x)] = composite_data['Avg_Row' + str(fb) + 'GTI_DNIReflected'] / Sum_DNI_ALB.m

#### Cell or Module Surface Temperature
The following will calculate the cell and module surface temperature using the King model as a default. Other models can be used as described at,  
https://pvlib-python.readthedocs.io/en/stable/reference/pv_modeling/temperature.html. The difference is less than one °C for ground mounted systems
but can be as high as 3 °C for a high temperature building integrated system.

Here the temperatures are added to the dataframe and the module temperature is selected as the default 'temperatue' for the degradation calculations. If it is a cell degradation that is being investigated, temp_cell should be used.

In [ ]:
temp_cell = pvdeg.temperature.cell(weather_df=weather_df, meta=meta, poa=poa_df)
temp_module = pvdeg.temperature.module(weather_df=weather_df, meta=meta, poa=poa_df)

weather_df['temp_cell'] = temp_cell
weather_df['temp_module'] = temp_module

weather_df['temperature'] = weather_df['temp_module']
#weather_df['temperature'] = weather_df['temp_cell']

#### Humidity
Depending on the component for which the calculation is being run on, the desired humidity may be the atmospheric humidity, the module surface humidity, the humidity in front of a cell with a permeable backsheet, the humidity in the backsheet, the humidity in the back encapsulant or another custom humidity location such as a diffusion limited location. The folowing are options for doing all of these calculations. Here all the different humidities are put in the weather_df dataframe, but to select one to be specifically used it should be named 'RH' for most degradation functions (check the documentation of a specific degradation calculation if in doubt). Here the surface humidity is selected as a default.

In [ ]:
# Calculate relative humidity at different locations in the module
RH_surface = pvdeg.humidity.surface_relative(weather_df['relative_humidity'], weather_df['temp_air'],temp_module)
RH_front_encapsulant = pvdeg.humidity.front_encapsulant(weather_df['relative_humidity'], weather_df['temp_air'],temp_cell,encapsulant='W001')
RH_back_encapsulant = pvdeg.humidity.back_encapsulant_water_concentration(
    temp_module = temp_module, 
    rh_surface = RH_surface,
    backsheet='W017',
    encapsulant='W001')
Ce_back_encapsulant = pvdeg.humidity.back_encapsulant_water_concentration(
    temp_module = temp_module, 
    rh_surface = RH_surface,
    backsheet='W017',
    encapsulant='W001', 
    output='ce')
RH_backsheet = (RH_surface + RH_back_encapsulant) / 2



Append the calculated values into the weather DataFrame.
Note: putting the values into the weather_df DataFrame is not strictly necessary, but may be convenient for later use in the degradation calculations.

In [ ]:
weather_df['RH_surface'] = RH_surface
weather_df['RH_front_encapsulant'] = RH_front_encapsulant
weather_df['Ce_back_encapsulant'] = Ce_back_encapsulant   
weather_df['RH_back_encapsulant'] = RH_back_encapsulant
weather_df['RH_backsheet'] = RH_backsheet

weather_df['poa_global'] = poa_df['poa_global']

Each of the necessary arrays of data can be individually sent to a function for calculation in the function call, or they can be combined into a single dataframe. The degradation functions are set up to first check for a specific data set in the function call but if not found it looks for specific data or a suitable substitute in the weather dataframe.

You can select one of the RH values to be used as the relative humidity in the degradation model calculations by assigning it to to column "RH" in the dataframe.
Alternatively, the "RH" data can be sent to the degradation function explicitly in the function call.

In [ ]:
weather_df['RH'] = RH_surface
#weather_df['RH'] =RH_front_encapsulant
#weather_df['RH'] = Ce_back_encapsulant      
#weather_df['RH'] = RH_back_encapsulant
#weather_df['RH'] = RH_backsheet

#display(weather_df)

## 2. Gather Basic Degradation Modeling Data for a Material of Interest

First we need to gather in the parameters for the degradation process of interest. This includes things such as the activiation energy and parameters defining the sensitivity to moisture, UV light, voltage, and other stressors.
For this tutorial we will need solar position, POA, PV cell and module temperature. Let's gernate those individually with their respective functions.
The blocked out text will produce a list of key fields from the database for each entry.

In [ ]:
#kwarg_variables = pvdeg.utilities._read_material(name=None, fname="DegradationDatabase", item=("Material", "Equation", "KeyWords", "EquationType"))
#print(json.dumps(kwarg_variables, skipkeys = True, indent = 0 ).replace("{" + "\n", "{").replace('\"' + "\n", "\"").replace(': {' , ':' + "\n" + "{").replace('},' + "\n", '},' +'\n' +'\n'))
pvdeg.utilities.display_json(pvdeg_file="DegradationDatabase", fp=DATA_DIR)

This next set of codes will take the data from the extracted portion of the Json library and create a list of variables from it. If more variables need to be modified or added, this is where it should be done.

In [ ]:
deg_data = pvdeg.utilities.read_material(fp=DATA_DIR, key="D036", pvdeg_file="DegradationDatabase")
print(json.dumps(deg_data, skipkeys = True, indent = 0 ).replace("{" + "\n", "{").replace('\"' + "\n", "\"").replace(': {' , ':' + "\n" + "{").replace('\n'+'}', '}'))

{"DataEntryPerson": "Michael Kempe",
"DateEntered": "2/14/2025",
"DOI": "10.1109/PVSC45281.2020.9300357",
"SourceTitle": "Highly Accelerated UV Stress Testing for Transparent Flexible Frontsheets",
"Authors": "Michael D Kempe, Peter Hacke, Joshua Morse, Michael Owen-Bellini, Derek Holsapple, Trevor Lockman, Samantha Hoang, David Okawa, Tamir Lance, Hoi Hong Ng",
"Reference": "Kempe, M. D., et al. (2020). Highly Accelerated UV Stress Testing for Transparent Flexible Frontsheets. 2020 47th IEEE Photovoltaic Specialists Conference (PVSC).",
"KeyWords": "Humidity, Irradiance, reciprocity",
"Material": "Flexible Frontsheet, Frontsheet Coatings",
"Degradation": "UV Cut On, UV Transmittance 310nm-350nm, Yellowness index, SPQEWT",
"EquationType": "arrhenius",
"Equation": "R_D=R_0\\cdot RH^n\\cdot G_{340}^P\\cdot e^{ \\left( \\frac{-E_a}{R\\cdot T_K } \\right) }",
"R_D":
{"units": "%/h"},
"R_0":
{"units": "%/h"},
"E_a":
{"value": 38.7,
"stdev": 21.7,
"units": "kJ/mol"},
"n":
{},
"p":
{"value": 

Here we pull out the relevant equation code identifier needed for running the calculations.

In [ ]:
func = "pvdeg.degradation." + deg_data["EquationType"]
print(func)
display(Math("\\Large " + deg_data["Equation"]))

pvdeg.degradation.arrhenius


<IPython.core.display.Math object>

## 3. Calculate Absolute Degradation Rate

To do this calculation, we must have degradation parameter data for a process that is complete with all the necessary variables. 

In [ ]:
from pvdeg import TEST_DATA_DIR
INPUT_SPECTRA = os.path.join(TEST_DATA_DIR, r"spectra_pytest.csv")
data = pd.read_csv(INPUT_SPECTRA)
display(data)
print(INPUT_SPECTRA)



,Unnamed: 0,RH,Temperature,"Spectra: [ 300, 325, 350, 375, 400 ]"
0,2021-01-01 00:00:00-05:00,95,-1.1,"[nan, nan, nan, nan, nan]"
1,2021-01-01 01:00:00-05:00,72,-1.1,"[nan, nan, nan, nan, nan]"
2,2021-01-01 02:00:00-05:00,72,-1.3,"[nan, nan, nan, nan, nan]"
3,2021-01-01 03:00:00-05:00,72,-1.5,"[nan, nan, nan, nan, nan]"
4,2021-01-01 04:00:00-05:00,72,-1.7,"[nan, nan, nan, nan, nan]"
...,...,...,...,...
8755,2021-12-31 19:00:00-05:00,92,-1.1,"[nan, nan, nan, nan, nan]"
8756,2021-12-31 20:00:00-05:00,92,-0.7,"[nan, nan, nan, nan, nan]"
8757,2021-12-31 21:00:00-05:00,92,-0.4,"[nan, nan, nan, nan, nan]"
8758,2021-12-31 22:00:00-05:00,92,0.0,"[nan, nan, nan, nan, nan]"


C:\Users\mkempe\Documents\GitHub\new\PVDegradationTools\tests\data\spectra_pytest.csv


## 3. VantHoff Degradation

Van 't Hoff Irradiance Degradation

For one year of degredation the controlled environmnet lamp settings will need to be set to IWa.

As with most `pvdeg` functions, the following functions will always require two arguments (weather_df and meta)

## 4. Arrhenius
Calculate the Acceleration Factor between the rate of degredation of a modeled environmnet versus a modeled controlled environmnet

Example: "If the AF=25 then 1 year of Controlled Environment exposure is equal to 25 years in the field"

Equation:
$$ AF = N * \frac{ I_{chamber}^x * RH_{chamber}^n * e^{\frac{- E_a}{k T_{chamber}}} }{ \Sigma (I_{POA}^x * RH_{outdoor}^n * e^{\frac{-E_a}{k T_outdoor}}) }$$

Function to calculate IWa, the Environment Characterization (W/m²). For one year of degredation the controlled environmnet lamp settings will need to be set at IWa.

Equation:
$$ I_{WA} = [ \frac{ \Sigma (I_{outdoor}^x * RH_{outdoor}^n e^{\frac{-E_a}{k T_{outdood}}}) }{ N * RH_{WA}^n * e^{- \frac{E_a}{k T_eq}} } ]^{\frac{1}{x}} $$

In [ ]:
# relative humidity within chamber (%)
rh_chamber = 15
# arrhenius activation energy (kj/mol)
Ea = 40
I_chamber = 1000  # irradiance within chamber (W/m^2)
temp_chamber = 25  # temperature within chamber (C)

rh_surface = pvdeg.humidity.surface_relative(rh_ambient=weather_df['relative_humidity'],
                                               temp_ambient=weather_df['temp_air'],
                                               temp_module=temp_module)

arrhenius_deg = pvdeg.degradation.arrhenius_deg(weather_df=weather_df, meta=meta,
                                                rh_outdoor=rh_surface,
                                                I_chamber=I_chamber,
                                                rh_chamber=rh_chamber,
                                                temp_chamber=temp_chamber,
                                                poa=poa_df,
                                                temp=temp_cell,
                                                Ea=Ea)

irr_weighted_avg_a = pvdeg.degradation.IwaArrhenius(weather_df=weather_df, meta=meta,
                                                    poa=poa_df,
                                                    rh_outdoor=weather_df['relative_humidity'],
                                                    temp=temp_cell,
                                                    Ea=Ea)

## 5. Quick Method (Degradation)

For quick calculations, you can omit POA and both module and cell temperature. The function will calculate these figures as needed using the available weather data with the default options for PV module configuration.

In [ ]:
# chamber settings
I_chamber= 1000
temp_chamber=60
rh_chamber=15

# activation energy
Ea = 40

vantHoff_deg = pvdeg.degradation.vantHoff_deg(weather_df=weather_df, meta=meta,
                                              I_chamber=I_chamber,
                                              temp_chamber=temp_chamber)

irr_weighted_avg_v = pvdeg.degradation.IwaVantHoff(weather_df=weather_df, meta=meta)

The array surface_tilt angle was not provided, therefore the latitude of  33.4 was used.
The array azimuth was not provided, therefore an azimuth of  180.0 was used.
The array surface_tilt angle was not provided, therefore the latitude of  33.4 was used.
The array azimuth was not provided, therefore an azimuth of  180.0 was used.


In [ ]:
rh_surface = pvdeg.humidity.surface_relative(rh_ambient=weather_df['relative_humidity'],
                                               temp_ambient=weather_df['temp_air'],
                                               temp_module=temp_module)

arrhenius_deg = pvdeg.degradation.arrhenius_deg(weather_df=weather_df, meta=meta,
                                                rh_outdoor=rh_surface,
                                                I_chamber=I_chamber,
                                                rh_chamber=rh_chamber,
                                                temp_chamber=temp_chamber,
                                                Ea=Ea)

irr_weighted_avg_a = pvdeg.degradation.IwaArrhenius(weather_df=weather_df, meta=meta,
                                                    rh_outdoor=weather_df['relative_humidity'],
                                                    Ea=Ea)

The array surface_tilt angle was not provided, therefore the latitude of  33.4 was used.
The array azimuth was not provided, therefore an azimuth of  180.0 was used.
The array surface_tilt angle was not provided, therefore the latitude of  33.4 was used.
The array azimuth was not provided, therefore an azimuth of  180.0 was used.


## 6. Solder Fatigue

Estimate the thermomechanical fatigue of flat plate photovoltaic module solder joints over the time range given using estimated cell temperature. Like other `pvdeg` funcitons, the minimal parameters are (weather_df, meta). Running the function with only these two inputs will use default PV module configurations ( open_rack_glass_polymer ) and the 'sapm' temperature model over the entire length of the weather data. 

In [ ]:
fatigue = pvdeg.fatigue.solder_fatigue(weather_df=weather_df, meta=meta)

The array surface_tilt angle was not provided, therefore the latitude of  33.4 was used.
The array azimuth was not provided, therefore an azimuth of  180.0 was used.


If you wish to reduce the span of time or use a non-default temperature model, you may specify the parameters manually. Let's try an explicit example.
We want the solder fatigue estimated over the month of June for a roof mounted glass-front polymer-back module.

1. Lets create a datetime-index for the month of June.
2. Next, generate the cell temperature. Make sure to explicity restrict the weather data to our dt-index for June. Next, declare the PV module configuration.
3. Calculate the fatigue. Explicity specify the time_range (our dt-index for June from step 1) and the cell temperature as we caculated in step 2

In [ ]:
# select the month of June
time_range = weather_df.index[weather_df.index.month == 6]

# calculate cell temperature over our selected date-time range.
# specify the module configuration
temp_cell = pvdeg.temperature.cell(weather_df=weather_df.loc[time_range], meta=meta,
                                   temp_model='sapm',
                                   conf='insulated_back_glass_polymer')


fatigue = pvdeg.fatigue.solder_fatigue(weather_df=weather_df, meta=meta,
                                       time_range = time_range,
                                       temp_cell = temp_cell)

The array surface_tilt angle was not provided, therefore the latitude of  33.4 was used.
The array azimuth was not provided, therefore an azimuth of  180.0 was used.
